## LOADING PRETRAINED MODEL

In [1]:
import pandas as pd

In [18]:
from simpletransformers.retrieval import RetrievalModel


model_type = "dpr"
model_name = "/home/rahvk/data/tmp/cache/model-hn-5/checkpoint-12844-epoch-1"

# Initialize a RetrievalModel
model = RetrievalModel(
    model_type=model_type,
    model_name=model_name,
)

## EVALUATING LOADED MODEL

In [19]:
train_data = pd.read_parquet('processed_df/train_data.parquet')


In [20]:
train_data.head()

,query_text,gold_passage,title,hard_negative
0,nikolaj coster waldau worked with the fox broa...,nikolaj coster waldau lrb lsb ne ola k sd ald ...,Nikolaj_Coster-Waldau,frederick fred seibert lrb born september 15 1...
1,nikolaj coster waldau was not in a danish thri...,nikolaj coster waldau lrb lsb ne ola k sd ald ...,Nikolaj_Coster-Waldau,lars von trier lrb lars trier 30 april 1956 rr...
2,nikolaj coster waldau worked with peter dinklage,nikolaj coster waldau lrb lsb ne ola k sd ald ...,Nikolaj_Coster-Waldau,arthur schopenhauer lrb lsb a t o pm ha rsb 22...
3,nikolaj coster waldau refused to ever work wit...,nikolaj coster waldau lrb lsb ne ola k sd ald ...,Nikolaj_Coster-Waldau,harry herbert frazee lrb june 29 1880 june 4 1...
4,nikolaj coster waldau was in a film,nikolaj coster waldau lrb lsb ne ola k sd ald ...,Nikolaj_Coster-Waldau,armin mueller stahl lrb born 17 december 1930 ...


In [21]:
result = model.eval_model(train_data)


Map:   0%|          | 0/102748 [00:00<?, ? examples/s]

Map:   0%|          | 0/102748 [00:00<?, ? examples/s]

Map:   0%|          | 0/102748 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/102748 [00:00<?, ? examples/s]

  0%|          | 0/103 [00:00<?, ?it/s]

Map:   0%|          | 0/102748 [00:00<?, ? examples/s]

Map:   0%|          | 0/102748 [00:00<?, ? examples/s]

Running Evaluation:   0%|          | 0/12844 [00:00<?, ?it/s]

/home/rahvk/data/tmp/anaconda3/envs/faiss_1.7.4/lib/python3.10/site-packages/simpletransformers/retrieval/retrieval_model.py:1659: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  (max_idxs == torch.tensor(labels)).sum().cpu().detach().numpy().item()


Retrieving docs:   0%|          | 0/201 [00:00<?, ?it/s]

In [22]:
res, doc_ids, doc_vectors, doc_dicts = result
res

{'eval_loss': 6.47824800401875,
 'mrr@1': 0.0,
 'mrr@2': 0.0,
 'mrr@3': 0.0,
 'mrr@5': 0.0,
 'mrr@10': 0.0,
 'top_1_accuracy': 0.0,
 'top_2_accuracy': 0.0,
 'top_3_accuracy': 0.0,
 'top_5_accuracy': 0.0,
 'top_10_accuracy': 0.0}

## MAKING PREDICTIONS

In [7]:
merged_df = pd.read_parquet('processed_df/merged_df.parquet')


In [8]:
merged_df.head()

,claim_id,label,claim,wiki_title,wiki_index,wiki_text
0,75397,SUPPORTS,nikolaj coster waldau worked with the fox broa...,Nikolaj_Coster-Waldau,3508521,nikolaj coster waldau lrb lsb ne ola k sd ald ...
1,58718,REFUTES,nikolaj coster waldau was not in a danish thri...,Nikolaj_Coster-Waldau,3508521,nikolaj coster waldau lrb lsb ne ola k sd ald ...
2,134655,SUPPORTS,nikolaj coster waldau worked with peter dinklage,Nikolaj_Coster-Waldau,3508521,nikolaj coster waldau lrb lsb ne ola k sd ald ...
3,86306,REFUTES,nikolaj coster waldau refused to ever work wit...,Nikolaj_Coster-Waldau,3508521,nikolaj coster waldau lrb lsb ne ola k sd ald ...
4,149361,SUPPORTS,nikolaj coster waldau was in a film,Nikolaj_Coster-Waldau,3508521,nikolaj coster waldau lrb lsb ne ola k sd ald ...


In [9]:
queries = list(set(merged_df['claim']))
passages = list(set(merged_df['wiki_text']))

In [10]:
to_predict = queries[:5]

In [11]:
predicted_passages, doc_ids, doc_vectors, doc_dicts = model.predict(to_predict, prediction_passages=passages)

Map:   0%|          | 0/9528 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9528 [00:00<?, ? examples/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Generating query embeddings: 0it [00:00, ?it/s]

Retrieving docs:   0%|          | 0/1 [00:00<?, ?it/s]

Note: predicted_passages is a list of lists having predicted top-10 documents per claim in the 'to_predict' list

In [12]:
print(len(predicted_passages))
print(len(predicted_passages[0]))

5
10


In [17]:
for i in range(5):
    claim = to_predict[i]
    print("Claim:", claim)
    print("Actual passage:", merged_df[merged_df['claim'] == claim]['wiki_text'])
    print("Predicted top passage:", predicted_passages[i][0])

Claim: david carradine had offenses
Actual passage: 29798    david carradine lrb born john arthur carradine...
Name: wiki_text, dtype: object
Predicted top passage: hell in many religious and folkloric traditions is a place of torment and punishment in an afterlife religions with a linear divine history often depict hells as eternal destinations while religions with a cyclic history often depict a hell as an intermediary period between incarnations typically these traditions locate hell in another dimension or under the earth s surface and often include entrances to hell from the land of the living other afterlife destinations include heaven purgatory paradise and limbo other traditions which do not conceive of the afterlife as a place of punishment or reward merely describe hell as an abode of the dead the grave a neutral place located under the surface of earth lrb for example see sheol and hades rrb hell is sometimes portrayed as populated with demons who torment those dwelling ther

## NOTE:

OUTPUT DOCUMENT FORMAT (JSON):

{"claim_id": ["wiki_title1", "wiki_title2", ...]}